In [2]:
from geopy.geocoders import Nominatim
import pandas as pd

In [3]:
app = Nominatim(user_agent="JournalDev")

In [256]:
csvfile = pd.read_csv('../data/raw/ltn-sensors.csv')

In [257]:
def geocode_ltn(row):
    import time
    import numpy
    try:
        time.sleep(1)
        location = app.geocode(row['Location'])
        # return 'Point(' + location.raw['lon'] + ' ' + location.raw['lat'] + ')'
        return [numpy.float64(location.raw['lon']), numpy.float64(location.raw['lat'])]
    except Exception as e:
        print(e)
        return None

csvfile = pd.concat([csvfile, csvfile.apply(geocode_ltn, axis=1,result_type='expand')], axis=1)
csvfile.rename(columns={0:"Longitude", 1:"Latitude"}, inplace=True)

csvfile



,Name,Location,Result,Longitude,Latitude
0,SSDT_19,193 George Ln London SE13 6RY,16.7,-0.007804,51.449719
1,SSDT_36,12 Pragnell Rd London SE12 0LF,17.4,0.020850,51.437360
2,SSDT_14,160 Leahurst Rd London SE13 5NL,17.6,0.000423,51.452562


In [274]:
csvfile.to_csv('../data/processed/ltn-sensors-geocoded.csv', index=False)

In [279]:
gdf = geopandas.read_file('../data/interim/ltn-sensors-geocoded.csv')

ValueError: GeoDataFrame does not support multiple columns using the geometry column name 'geometry'.

In [278]:
gdf

,Name,Location,Result,Longitude,Latitude,coordinates,geometry
0,SSDT_19,193 George Ln London SE13 6RY,16.7,-0.0078041,51.4497192,POINT (-0.0078041 51.4497192),None
1,SSDT_36,12 Pragnell Rd London SE12 0LF,17.4,0.0208502,51.4373605,POINT (0.0208502 51.4373605),None
2,SSDT_14,160 Leahurst Rd London SE13 5NL,17.6,0.0004235,51.4525616,POINT (0.0004235 51.4525616),None


In [265]:
import geopandas 
csvfile['geometry'] = geopandas.points_from_xy(csvfile.Longitude, csvfile.Latitude)

In [266]:
csvfile

,Name,Location,Result,Longitude,Latitude,geometry
0,SSDT_19,193 George Ln London SE13 6RY,16.7,-0.007804,51.449719,POINT (-0.00780 51.44972)
1,SSDT_36,12 Pragnell Rd London SE12 0LF,17.4,0.020850,51.437360,POINT (0.02085 51.43736)
2,SSDT_14,160 Leahurst Rd London SE13 5NL,17.6,0.000423,51.452562,POINT (0.00042 51.45256)


In [273]:
import geopandas as gpd
# gdf = gpd.GeoDataFrame(csvfile, crs="EPSG:4326", geometry=csvfile.geometry)


# print(gdf.head())

TypeError: 'DataFrame' object is not callable